### Hugging Face Inference API 활용하기

- **Hugging Face**는 무료 모델과 데이터셋을 공유 가능한 오픈소스 플랫폼
- 다양한 머신러닝 애플리케이션을 구축하고 배포할 수 있으며, 이를 위한 엔드포인트 서비스도 제공
- **엔드포인트**란 사용자가 원격에서 머신러닝 모델을 호출하고 사용할 수 있도록 설정된 URL 또는 API 인터페이스
- 서버리스 엔드포인트와 Inference API는 이러한 원격 호출과 배포를 간소화해 주는 서비스
- **서버리스 엔드포인트**는 사용자가 직접 관리할 필요 없이 모델을 호스팅하고 자동 확장 할 수 있는 API
- **Inference API**는 Hugging Face에서 제공하는 사전 학습된 모델을 쉽게 사용할 수 있도록 하는 API 서비스

In [1]:
from dotenv import load_dotenv

from langchain_teddynote import logging

load_dotenv()
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


- 모델 웹 페이지의 Chat Format에서 모델의 프롬프트 양식과 예시를 확인한다
- 모델마다 학습된 프롬프트 양식이 다르므로 정해진 양식을 사용하면 모델이 기대하는 형식에 맞춰 정확한 응답을 얻을 수 있다

```
<|system|>
You are a helpful assistant.<|end|>
<|user|>
Question?<|end|>
<|assistant|>
```

In [2]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer:"""

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate.from_template(template)

**Serverless Endpoints**

- Inference API 는 무료로 사용할 수 있으며 요금은 제한
- 프로덕션을 위한 추론 솔루션이 필요한 경우, [Inference Endpoints](https://huggingface.co/docs/inference-endpoints/index) 서비스를 확인
- Inference Endpoints 를 사용하면 모든 머신 러닝 모델을 전용 및 완전 관리형 인프라에 손쉽게 배포 가능.
- 클라우드, 지역, 컴퓨팅 인스턴스, 자동 확장 범위 및 보안 수준을 선택하여 모델, 지연 시간, 처리량 및 규정 준수 요구 사항에 맞게 설정
- 다음은 Inference API 에 액세스하는 방법의 예시
- 참고
  - [Serverless Endpoints](https://huggingface.co/docs/api-inference/index)
  - [Inference Endpoints](https://huggingface.co/docs/inference-endpoints/index)


- `repo_id` 변수에 HuggingFace 모델의 `repo ID`(저장소 ID) 를 할당
  - `microsoft/Phi-3-mini-4k-instruct` 모델: https://huggingface.co/microsoft/Phi-3-mini-4k-instruct

In [ ]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEndpoint

# 사용할 모델의 저장소 ID를 설정합니다.
repo_id = "microsoft/Phi-3-mini-4k-instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=256,  # 생성할 최대 토큰 길이를 설정합니다.
    temperature=0.1,
    task="text-generation",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],  # 허깅페이스 토큰
)

# LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
chain = prompt | llm | StrOutputParser()
# 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.
response = chain.invoke({"question": "what is the capital of South Korea?"})
print(response)


### Dedicated Inference Endpoint로 원격 호스팅하기

- 앞서 살펴본 서버리스 엔드포인트와 Inference API의 고급 버전
- 더 높은 성능과 안정성이 필요하다면 전용 서버 환경에서 모델을 직접 호스팅하는 **Dedicated Inference Endpoint** 고려

- `hf_endpoint_url` 변수에 Inference Endpoint의 URL을 설정

In [5]:
# Inference Endpoint URL을 아래에 설정합니다.
hf_endpoint_url = "https://slcalzucia3n7y3g.us-east-1.aws.endpoints.huggingface.cloud"

In [7]:
llm = HuggingFaceEndpoint(
    # 엔드포인트 URL을 설정합니다.
    endpoint_url=hf_endpoint_url,
    max_new_tokens=512,
    temperature=0.01,
    task="text-generation",
)

# 주어진 프롬프트에 대해 언어 모델을 실행합니다.
llm.invoke(input="대한민국의 수도는 어디인가요?")

AttributeError: 'InferenceClient' object has no attribute 'post'

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "A chat between a curious user and an artificial intelligence assistant. "
            "The assistant gives helpful, detailed, and polite answers to the user's questions.",
        ),
        ("user", "Human: {question}\nAssistant: "),
    ]
)

chain = prompt | llm | StrOutputParser()